In [ ]:
#Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import datetime
import scipy.stats as st
from scipy.stats import linregress
from pprint import pprint
from datetime import timezone
# Import API key
from config import weather_api_key

In [ ]:
path = "Data\storm_data.csv"
load = pd.read_csv(path)
hurricane_df = pd.DataFrame(load)
hurricane_df

In [ ]:
# Creating series to add to our dataframe
start_dates = []
unix_start_dates = []
unix_end_dates = []
end_dates = []
unix_compliant = []

# Array of tuples to reference months
months = [("Jan", 1), ("Feb", 2), ("Mar", 3), 
          ("Apr", 4), ("May", 5), ("Jun", 6), 
          ("Jul", 7), ("Aug", 8), ("Sep", 9), 
          ("Oct", 10), ("Nov", 11), ("Dec", 12)]

# Substrings of start dates and end dates
for date in hurricane_df["DATE RANGE"]:
    start_dates.append(date[0:12])
    end_dates.append(date[16:])

# Convert substrings into numeric values for start dates and convert them into unix timestamps
for date in start_dates:
    year = int(date[-4:])
    month = date[0:3]
    count = 0
    for x in months:
        if month == months[count][0]:
            month = months[count][1]
        else:
            count += 1
    day = int(date[4:6])
    dt = datetime.datetime(year, month, day)
    timestamp = dt.replace(tzinfo=timezone.utc).timestamp()
    timestamp = dt.replace(tzinfo=timezone.utc).timestamp()
    unix_start_dates.append(timestamp)
    
    # Categorizes dates into before 1970 or after 1970
    if year >= 1970:
        unix_compliant.append("True")
    else:
        unix_compliant.append("False")
        
# Convert substrings into numeric values for end dates and convert them into unix timestamps
for date in end_dates:
    year = int(date[-4:])
    month = date[0:3]
    count = 0
    for x in months:
        if month == months[count][0]:
            month = months[count][1]
        else:
            count += 1
    day = int(date[4:6])
    dt = datetime.datetime(year, month, day)
    timestamp = dt.replace(tzinfo=timezone.utc).timestamp()
    unix_end_dates.append(timestamp)

In [ ]:
# Adds series to dataframe
hurricane_df["START DATE"] = unix_start_dates
hurricane_df["END DATE"] = unix_end_dates
hurricane_df["UNIX COMPLIANT"] = unix_compliant

In [ ]:
# Filters data to be UNIX timestamp compliant
hurricane_df = hurricane_df[hurricane_df["UNIX COMPLIANT"] == "True"]
hurricane_df = hurricane_df[["STORM NAME & YEAR", "DATE RANGE", "START DATE", "END DATE", "MAX WIND SPEED", "MAX CATEGORY"]]

In [ ]:
# Change UNIX timestamps to fit the API parameter
hurricane_df["START DATE"]=(hurricane_df["START DATE"].astype(int))
hurricane_df["END DATE"]=(hurricane_df["END DATE"].astype(int))
hurricane_df

In [ ]:
# Drop dates that were before 1979 to fit the API call
hurricane_df = hurricane_df.drop(hurricane_df.index[32:])
hurricane_df

In [ ]:
#OpenWeather AP Extraction

url = " https://api.openweathermap.org/data/3.0/onecall/timemachine?"
latitude = 28.53
longitiude = -81.37

#testing calls
query_url = f"{url}lat={latitude}&lon={longitiude}&dt={1128470400}&appid={weather_api_key}"
query_url

In [ ]:
# Test if API worked
response = requests.get(query_url).json()
pprint(response)

In [ ]:
# response["data"][0]["pressure"]
response["data"][0]["weather"][0]["description"]

In [ ]:
#Set up lists to hold info
dt = []
temp = []
pressure = []
perc_humidity = []
perc_cloud = []
weather_id = []

for x in range(len(hurricane_df)):
    response = requests.get(f"{url}lat={latitude}&lon={longitiude}&dt={hurricane_df['START DATE'][x]}&appid={weather_api_key}").json()
    temp.append(response["data"][0]["temp"])
    pressure.append(response["data"][0]["pressure"])
    perc_humidity.append(response["data"][0]["humidity"])
    perc_cloud.append(response["data"][0]["clouds"])
    weather_id.append(response["data"][0]["weather"][0]["description"])
    dt.append(hurricane_df['START DATE'][x])

In [ ]:
# Dictionary for Weather Data on the Start Date
first_dict = {
            "START DATE": dt,
            "Start Temperature": temp,
            "Start Pressure": pressure,
            "Start Humidity(%)": perc_humidity,
            "Start Cloudiness(%)": perc_cloud,
            "Start Weather Description": weather_id
           }
first_df = pd.DataFrame(first_dict)
first_df.head(3)

In [ ]:
#Set up lists to hold info
dt = []
temp = []
pressure = []
perc_humidity = []
perc_cloud = []
weather_id = []

# def orlando_hurricane_info(dt):
#     response = requests.get(query_url).json()
for x in range(len(hurricane_df)):
    response = requests.get(f"{url}lat={latitude}&lon={longitiude}&dt={hurricane_df['END DATE'][x]}&appid={weather_api_key}").json()
    temp.append(response["data"][0]["temp"])
    pressure.append(response["data"][0]["pressure"])
    perc_humidity.append(response["data"][0]["humidity"])
    perc_cloud.append(response["data"][0]["clouds"])
    weather_id.append(response["data"][0]["weather"][0]["description"])
    dt.append(hurricane_df['END DATE'][x])

In [ ]:
# Dictionary for weather data for End Date
second_dict = {
            "END DATE": dt,
            "End Temperature": temp,
            "End Pressure": pressure,
            "End Humidity(%)": perc_humidity,
            "End Cloudiness(%)": perc_cloud,
            "End Weather Description": weather_id
           }
second_df = pd.DataFrame(second_dict)
second_df.head(3)

In [ ]:
# Merge main dataframe with collected weather data for Start and End Dates
merged_df1 = hurricane_df.merge(first_df, how="inner", on="START DATE")
merged_df1

In [ ]:
merged_df2 = merged_df1.merge(second_df, how="inner", on="END DATE")
merged_df2.head()

In [ ]:
# Reorder Columns
clean_df = merged_df2[["STORM NAME & YEAR", "DATE RANGE", "MAX CATEGORY", "MAX WIND SPEED", "Start Temperature", "End Temperature", "Start Pressure", "End Pressure",
                      "Start Humidity(%)", "End Humidity(%)", "Start Cloudiness(%)", "End Cloudiness(%)", "Start Weather Description", "End Weather Description"]]
clean_df

In [ ]:
# Write Cleaned Data to a CSV file
clean_df.to_csv("Orlando Hurricane Data.csv")